In [1]:
import finlab

In [2]:
finlab.login(api_token='ID0hv1JVLP3bs86Qg/xslT1fwKjN0bclQVRCH7eQiAYh3e5/XNh77u3zaPDOOGJQ#vip_m')

輸入成功!


In [3]:
from finlab import data
from finlab.backtest import sim
import pandas as pd
import numpy as np
from finlab import data
from finlab.backtest import sim
import pandas as pd
import numpy as np
from finlab import backtest

In [4]:
operating_income = data.get('financial_statement:營業收入淨額')
operating_profit = data.get('financial_statement:營業利益')
operating_profit_ratio = (operating_profit / operating_income) * 100

def get_return_by_seasons(seasons):
  ret = {}
  return_dataset = {}
  for drop in range(-50, 10, 10):
    condition1 = (((operating_profit_ratio - operating_profit_ratio.shift(1)) / abs(operating_profit_ratio.shift(1))).rolling(seasons-1).min()) * 100 > drop
    condition2 = operating_profit_ratio.rolling(seasons).min() > 0
    position = condition1 & condition2
    report = backtest.sim(position.loc['2014':], resample="M", upload=False)
    return_series = report.creturn
    return_dataset['seasons'] = seasons
    return_dataset[f"{drop}"] = report.creturn
  return_dataset = pd.DataFrame(return_dataset)
  return return_dataset

return_season_df = pd.concat([get_return_by_seasons(s) for s in [8, 7, 6, 5, 4, 3, 2]])



 Use "pip install finlab==0.4.5" to update the latest version.


In [5]:
close = data.get('price:收盤價')
rev = data.get('monthly_revenue:當月營收')
yoy = data.get("monthly_revenue:去年同月增減(%)")

def get_return_by_percentage(drop):
  condition1 = (((operating_profit_ratio - operating_profit_ratio.shift(1)) / abs(operating_profit_ratio.shift(1))).rolling(3).min()) * 100 > drop
  condition2 = operating_profit_ratio.rolling(4).min() > 0
  #condition4 = yoy.average(6) > 0
  condition4 = rev >= rev.rolling(6).max()

  price_range = np.arange(0, 60, 10)
  ret = []
  for s,e in zip(price_range, price_range[1:]):
    condition3 = (close > s) & (close < e)
    position = condition1 & condition2 & condition3 & condition4
    r = backtest.sim(position.loc['2014':], resample="M", upload=False)
    return_series = r.creturn
    ret.append({'%':drop, 'price_range':f"{round(s,1)}-{round(e,1)}", 'return':return_series[-1]})
  df = pd.DataFrame(ret)
  return df

return_percentage_df = pd.concat([get_return_by_percentage(percentage) for percentage in np.linspace(-5.0, 5.0, 20)])

In [6]:
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [8]:
condition1 = (((operating_profit_ratio - operating_profit_ratio.shift(1)) / abs(operating_profit_ratio.shift(1))).rolling(3).min()) * 100 > 4
condition2 = operating_profit_ratio.rolling(4).min() > 0
# condition3 = (close > 30) & (close < 40)

buy = condition1 & condition2
report = backtest.sim(buy.loc['2018':], resample="M", upload=False)
report.benchmark = data.get('benchmark_return:發行量加權股價報酬指數').squeeze()
report.display()

--Stats--
annualized_rate_of_return      17.1%
sharpe                          0.69
max_drawdown                 -42.36%
win_ratio                     57.55%

--Latest Position--


stock_id
1319    0.035714
1473    0.035714
1580    0.035714
1616    0.035714
2421    0.035714
2453    0.035714
2633    0.035714
2707    0.035714
2903    0.035714
3680    0.035714
4119    0.035714
4207    0.035714
4439    0.035714
4741    0.035714
4766    0.035714
5284    0.035714
5469    0.035714
5511    0.035714
6170    0.035714
6235    0.035714
6244    0.035714
6291    0.035714
6443    0.035714
6523    0.035714
6664    0.035714
6703    0.035714
8996    0.035714
9914    0.035714
Name: 2023-05-31 00:00:00, dtype: float64

In [9]:
rev = data.get('monthly_revenue:當月營收')

condition1 = (((operating_profit_ratio - operating_profit_ratio.shift(1)) / abs(operating_profit_ratio.shift(1))).rolling(3).min()) * 100 > -1
condition2 = operating_profit_ratio.rolling(4).min() > 0
condition3 = (close > 30) & (close < 40)
condition4 = operating_profit_ratio > operating_profit_ratio.shift(1)
condition5 = rev >= rev.rolling(6).max()#rev.average(3) > rev.average(12)

buy = condition1 & condition2 & condition3 & condition4 & condition5
report = backtest.sim(buy.loc['2014':], resample="M", upload=False)
report.benchmark = data.get('benchmark_return:發行量加權股價報酬指數').squeeze()
report.display()

--Stats--
annualized_rate_of_return     51.37%
sharpe                          1.62
max_drawdown                 -40.57%
win_ratio                     62.87%

--Latest Position--


Series([], Name: 2023-07-31 00:00:00, dtype: float64)

In [ ]:
import finlab
finlab.login(api_token='ID0hv1JVLP3bs86Qg/xslT1fwKjN0bclQVRCH7eQiAYh3e5/XNh77u3zaPDOOGJQ#vip_m')
from finlab import data
from finlab.backtest import sim
import pandas as pd
import numpy as np
from finlab import backtest

with data.universe(market='TSE_OTC'): 
  rev = data.get('monthly_revenue:當月營收')
  operating_income = data.get('financial_statement:營業收入淨額')
  operating_profit = data.get('financial_statement:營業利益')
  close = data.get('price:收盤價')
  mv = 股本 = data.get('financial_statement:股本')*close
  increasing_share = data.get('internal_equity_changes:董監增加股數')

low_market = mv < mv.quantile_row(0.5)

operating_profit_ratio = (operating_profit / operating_income) * 100

condition1 = (((operating_profit_ratio - operating_profit_ratio.shift(1)) / abs(operating_profit_ratio.shift(1))).rolling(3).min()) * 100 > 5
condition2 = operating_profit_ratio.rolling(4).min() > 0
condition3 = (close > 30) & (close < 40)
condition4 = operating_profit_ratio.rise(4)# > operating_profit_ratio.shift(1)
condition5 = rev >= rev.rolling(6).max()
condition6 = low_market

buy = condition1 & condition2 & condition3 & condition4 & condition5
# report = backtest.sim(buy.loc['2014':], resample="M",name='營業利益率突破', stop_loss=0.25)
buy = buy[buy.index >= '2018-09-01']
report = sim(position=buy,resample="M", stop_loss=0.2, position_limit=0.25, fee_ratio=1.425/1000*0.3,name="高勝率營利率策略",upload=True, mae_mfe_window=40)
print(report.get_stats(resample='1d', riskfree_rate=0.02)['daily_sharpe'])
print(report.get_stats(resample='1d', riskfree_rate=0.02)['max_drawdown'])
report.display()
report.display_mae_mfe_analysis()

輸入成功!


1.4248714775412286
-0.1831242832485117


--Stats--
annualized_rate_of_return     27.42%
sharpe                          1.42
max_drawdown                 -18.31%
win_ratio                     65.57%

--Latest Position--


Series([], Name: 2023-05-31 00:00:00, dtype: float64)

In [ ]:
report.trades['return'].max()

1.2621359223300965